<a href="https://colab.research.google.com/github/flydragon2018/colab_notebooks/blob/main/image_grainsize_cv_measure_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
! unzip -q  /content/drive/MyDrive/hw/2021hwjx-data-train.zip -d /content/

In [ ]:
g_avgarea=[0.2581,0.1290,0.0912,0.0645,0.0456,0.0323,0.0228,0.0161,0.0114,0.00806,0.00570,0.00403,0.00285,0.00202,0.00143,0.00101,0.00071,0.00050,0.00036,0.00025,0.00018,0.00013,0.000089,0.000063,0.000045,0.000032,0.000022,0.000016,0.000011,0.000008]
g_avgdiameter=[0.5080,0.3592,0.3021,0.2540,0.2136,0.1796,0.1510,0.1270,0.1068,0.0898,0.0755,0.0635,0.0534,0.0449,0.0378,0.0318,0.0267,0.0225,0.0186,0.0159,0.0133,0.0112,0.0094,0.0079,0.0067,0.0056,0.0047,0.0040,0.0033,0.0028]

In [ ]:
#g_names=['00','0','0.5','1.0','1.5','2.0','2.5','3.0','3.5','4.0','5.0','5.5','6.0','6.5','7.0','7.5','8.0','8.5','9.0','9.5','10.0','10.5','11.0','11.5','12.0','12.5','13.0','13.5','14.0']


In [ ]:
g_names=['6.5','7.0','7.5','8.0','8.5','9.0','9.5','10.0','10.5','11.0','11.5','12.0','12.5','13.0']


In [ ]:
g_areas=[0.00143,0.00101,0.00071,0.00050,0.00036,0.00025,0.00018,0.00013,0.000089,0.000063,0.000045,0.000032,0.000022,0.000016]
g_diameters=[0.0378,0.0318,0.0267,0.0225,0.0186,0.0159,0.0133,0.0112,0.0094,0.0079,0.0067,0.0056,0.0047,0.0040]

In [ ]:
##   G=-6.643856*lg(avg_diameter)-3.288

In [ ]:
import cv2
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from skimage import io, color, measure 
import os

In [ ]:
'''

# -*- coding: utf-8 -*-
import os
import torch
import torchvision

from model_service.pytorch_model_service import PTServingBaseService

import time
from metric.metrics_manager import MetricsManager
from torch import nn
import torch.nn.functional as F
from torchvision import transforms

import log
from PIL import Image

Image.MAX_IMAGE_PIXELS = 1000000000000000
logger = log.getLogger(__name__)

logger.info(torch.__version__)
logger.info(torchvision.__version__)


class ImageClassificationService(PTServingBaseService):
    def __init__(self, model_name, model_path, **kwargs):
        self.model_name = model_name
        self.model_path = model_path
        self.labels = self.read_classes()
        self.num_classes = len(self.labels)
        logger.info('{}-{}'.format(self.num_classes, self.model_path))
        for key in kwargs:
            logger.info('{}-{}'.format(key, kwargs[key]))

        self.model = self.initial_model(self.num_classes)

        self.use_cuda = False
        self.device = torch.device("cuda"
                                   if torch.cuda.is_available() else "cpu")
        if torch.cuda.is_available():
            logger.info('Using GPU for inference')
            self.model.to(self.device)
            self.model.load_state_dict(torch.load(self.model_path)['model'])
        else:
            logger.info('Using CPU for inference')
            self.model.load_state_dict(torch.load(self.model_path,
                                                  map_location='cpu')['model'])
        self.model.eval()

    def initial_model(self, num_classes):
        model = torchvision.models.__dict__["resnet50"](pretrained=False)
        channel_in = model.fc.in_features
        model.fc = nn.Linear(channel_in, num_classes)
        logger.info('{}-{}'.format(channel_in, num_classes))
        return model

    # transform img before inference
    def transform_img(self, img):
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize])
        return transform(img)

    def read_classes(self):
        labels = []
        class_path = os.path.join(os.path.dirname(self.model_path), 'classes.txt')
        with open(class_path) as lines:
            for line in lines:
                line = line.strip()
                labels.append(line)
        return labels

    def _preprocess(self, data):
        preprocessed_data = {}
        for k, v in data.items():
            for file_name, file_content in v.items():
                img = Image.open(file_content).convert("RGB")
                img = self.transform_img(img)
                preprocessed_data[k] = torch.unsqueeze(img, dim=0).to(self.device)
        return preprocessed_data

    def _inference(self, data):
        img = data["input_img"]
        data = img
        res = self.model(data)
        res_softmax = F.softmax(res[0],dim=0).to('cpu')
        conf, pred = res_softmax.topk(1, 0, True, True)
        label = self.labels[pred.item()]
        result = {}
        result['label'] = label
        result['confidence'] = float('{0:.4f}'.format(conf.item()))
        result = {"result": result}
        return result

    def _postprocess(self, data):
        return data

    def inference(self, data):
        pre_start_time = time.time()
        data = self._preprocess(data)
        infer_start_time = time.time()
        # Update preprocess latency metric
        pre_time_in_ms = (infer_start_time - pre_start_time) * 1000
        logger.info('preprocess time: ' + str(pre_time_in_ms) + 'ms')
        if self.model_name + '_LatencyPreprocess' in MetricsManager.metrics:
            MetricsManager.metrics[self.model_name + '_LatencyPreprocess'].update(pre_time_in_ms)
        data = self._inference(data)
        infer_end_time = time.time()
        infer_in_ms = (infer_end_time - infer_start_time) * 1000
        logger.info('infer time: ' + str(infer_in_ms) + 'ms')
        data = self._postprocess(data)
        # Update inference latency metric
        post_time_in_ms = (time.time() - infer_end_time) * 1000
        logger.info('postprocess time: ' + str(post_time_in_ms) + 'ms')
        if self.model_name + '_LatencyInference' in MetricsManager.metrics:
            MetricsManager.metrics[self.model_name + '_LatencyInference'].update(post_time_in_ms)
        # Update overall latency metric
        if self.model_name + '_LatencyOverall' in MetricsManager.metrics:
            MetricsManager.metrics[self.model_name + '_LatencyOverall'].update(pre_time_in_ms + post_time_in_ms)
        logger.info('latency: ' + str(pre_time_in_ms + infer_in_ms + post_time_in_ms) + 'ms')
        data['latency_time'] = pre_time_in_ms + infer_in_ms + post_time_in_ms
        return data


'''

'\n\n# -*- coding: utf-8 -*-\nimport os\nimport torch\nimport torchvision\n\nfrom model_service.pytorch_model_service import PTServingBaseService\n\nimport time\nfrom metric.metrics_manager import MetricsManager\nfrom torch import nn\nimport torch.nn.functional as F\nfrom torchvision import transforms\n\nimport log\nfrom PIL import Image\n\nImage.MAX_IMAGE_PIXELS = 1000000000000000\nlogger = log.getLogger(__name__)\n\nlogger.info(torch.__version__)\nlogger.info(torchvision.__version__)\n\n\nclass ImageClassificationService(PTServingBaseService):\n    def __init__(self, model_name, model_path, **kwargs):\n        self.model_name = model_name\n        self.model_path = model_path\n        self.labels = self.read_classes()\n        self.num_classes = len(self.labels)\n        logger.info(\'{}-{}\'.format(self.num_classes, self.model_path))\n        for key in kwargs:\n            logger.info(\'{}-{}\'.format(key, kwargs[key]))\n\n        self.model = self.initial_model(self.num_classes)\n

In [ ]:
def __init__(self, model_name, model_path, **kwargs):
    self.model_name = model_name
    self.model_path = model_path
    self.labels = self.read_classes()
    self.num_classes = len(self.labels)
    logger.info('{}-{}'.format(self.num_classes, self.model_path))
    for key in kwargs:
        logger.info('{}-{}'.format(key, kwargs[key]))

    self.model = self.initial_model(self.num_classes)

    self.use_cuda = False
    self.device = torch.device("cuda"
                               if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        logger.info('Using GPU for inference')
        self.model.to(self.device)
        self.model.load_state_dict(torch.load(self.model_path)['model'])
    else:
        logger.info('Using CPU for inference')
        self.model.load_state_dict(torch.load(self.model_path,
                                              map_location='cpu')['model'])
    self.model.eval()

def initial_model(self, num_classes):
    model = torchvision.models.__dict__["resnet50"](pretrained=False)
    channel_in = model.fc.in_features
    model.fc = nn.Linear(channel_in, num_classes)
    logger.info('{}-{}'.format(channel_in, num_classes))
    return model

# transform img before inference
def transform_img(self, img):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize])
    return transform(img)

def read_classes(self):
    labels = []
    class_path = os.path.join(os.path.dirname(self.model_path), 'classes.txt')
    with open(class_path) as lines:
        for line in lines:
            line = line.strip()
            labels.append(line)
    return labels

def _preprocess(self, data):
    preprocessed_data = {}
    for k, v in data.items():
        for file_name, file_content in v.items():
            img = Image.open(file_content).convert("RGB")
            img = self.transform_img(img)
            preprocessed_data[k] = torch.unsqueeze(img, dim=0).to(self.device)
    return preprocessed_data

def _inference(self, data):
    img = data["input_img"]
    data = img
    #res = self.model(data)
    #res_softmax = F.softmax(res[0],dim=0).to('cpu')
    #conf, pred = res_softmax.topk(1, 0, True, True)
    #label = self.labels[pred.item()]
    labels=['6.5','7.0','7.5','8.0','8.5','9.0','9.5','10.0','10.5','11.0','11.5','12.0','12.5','13.0']
    pix2mm=25.4/96
    M=500
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh=cv2.threshold(img,0,255,cv2.THRESH_OTSU)
    kernel=np.ones((3,3),np.uint8)
    eroded=cv2.erode(thresh,kernel,iterations=5)
    dilated=cv2.dilate(eroded,kernel,iterations=2)  

    mask=dilated==255
    s=[[1,1,1],[1,1,1],[1,1,1]]
    

    labeled_mask,num_labels=ndimage.label(mask,structure=s)
    clusters=measure.regionprops(labeled_mask)
    d=[]
    for props in clusters:
      d.append(props['equivalent_diameter'])
    md=np.median(d)
    gmd=md*pix2mm/M
    err_d=list(abs(g_diameters-gmd))
    idx=err_d.index(np.min(err_d))


    result = {}
    result['label'] = labels[idx]
    #result['confidence'] = float('{0:.4f}'.format(conf.item()))
    result = {"result": result}
    return result

def _postprocess(self, data):
    return data

def inference(self, data):
    pre_start_time = time.time()
    data = self._preprocess(data)
    infer_start_time = time.time()
    # Update preprocess latency metric
    pre_time_in_ms = (infer_start_time - pre_start_time) * 1000
    logger.info('preprocess time: ' + str(pre_time_in_ms) + 'ms')
    if self.model_name + '_LatencyPreprocess' in MetricsManager.metrics:
        MetricsManager.metrics[self.model_name + '_LatencyPreprocess'].update(pre_time_in_ms)
    data = self._inference(data)
    infer_end_time = time.time()
    infer_in_ms = (infer_end_time - infer_start_time) * 1000
    logger.info('infer time: ' + str(infer_in_ms) + 'ms')
    data = self._postprocess(data)
    # Update inference latency metric
    post_time_in_ms = (time.time() - infer_end_time) * 1000
    logger.info('postprocess time: ' + str(post_time_in_ms) + 'ms')
    if self.model_name + '_LatencyInference' in MetricsManager.metrics:
        MetricsManager.metrics[self.model_name + '_LatencyInference'].update(post_time_in_ms)
    # Update overall latency metric
    if self.model_name + '_LatencyOverall' in MetricsManager.metrics:
        MetricsManager.metrics[self.model_name + '_LatencyOverall'].update(pre_time_in_ms + post_time_in_ms)
    logger.info('latency: ' + str(pre_time_in_ms + infer_in_ms + post_time_in_ms) + 'ms')
    data['latency_time'] = pre_time_in_ms + infer_in_ms + post_time_in_ms
    return data


In [ ]:
scipy.__version__